<a href="https://colab.research.google.com/github/pratikbhongade/AchPub/blob/main/NEWValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 10.5 MB/s eta 0:00:00


In [71]:
import pandas as pd
import re
import xlsxwriter
from tabulate import tabulate

# Open the ACH file and read the lines
with open('WrongNacha.ach', 'r') as f:
    lines = f.readlines()

# Create a DataFrame for each record type
header_df = pd.DataFrame([{
    'Record Type': lines[0][0:1],
    'Priority Code': lines[0][1:3],
    'Immediate Destination': lines[0][3:13],
    'Immediate Origin': lines[0][13:23],
    'File Creation Date': f"{lines[0][23:25]}/{lines[0][25:27]}/{lines[0][27:29]}",
    'File Creation Time': f"{lines[0][29:31]}:{lines[0][31:33]}",
    'File ID Modifier': lines[0][33:34],
    'Record Size': lines[0][34:37],
    'Blocking Factor': lines[0][37:39],
    'Format Code': lines[0][39:40],
    'Immediate Destination Name': lines[0][40:63],
    'Immediate Origin Name': lines[0][63:86],
    'Reference Code': lines[0][86:94]
}], index=[0])


# Create a DataFrame to store the batch header record entries
batch_header_df = pd.DataFrame(columns=['Record Type', 'Service Class Code', 'Company Name',
                                        'Company Identification', 'Standard Entry Class Code',
                                        'Company Entry Description', 'Company Descriptive Date',
                                        'Effective Entry Date', 'Settlement Date', 'Originator Status Code',
                                        'Originating DFI Identification', 'Batch Number'])

# Iterate over the lines in the file and populate the DataFrames

from datetime import datetime, timedelta
import datetime

for i in range(0, len(lines), 1):
    if lines[i][0:1] == '5':
        # Map of Standard Entry Class Codes to their descriptions
        sec_code_map = {
            'PPD': 'Prearranged Payment and Deposit Entry',
            'CCD': 'Corporate Credit or Debit',
            'CTX': 'Corporate Trade Exchange',
            'WEB': 'Internet-initiated Entry',
            'TEL': 'Telephone-initiated Entry',
            'RCK': 'Re-presented Check Entry',
            'POP': 'Point of Purchase Entry',
            'ARC': 'Accounts Receivable Entry',
            'BOC': 'Back Office Conversion Entry',
            'MTE': 'Machine Transfer Entry',
            'POS': 'Point of Sale Entry',
            'SHR': 'Shared Network Entry',
            'ADV': 'Automated Accounting Advice'
        }
        
        sec_code = lines[i][50:53]
        sec_desc = sec_code_map.get(sec_code, 'Unknown')
        
        batch_header_df = batch_header_df.append({
            'Record Type': lines[i][0:1],
            'Service Class Code': lines[i][1:4],
            'Company Name': lines[i][4:20],
            'Company Discretionary Data': lines[i][20:40],
            'Company Identification': lines[i][40:50],
            'Standard Entry Class Code': sec_code + ':' + sec_desc,
            'Company Entry Description': lines[i][53:63],
            'Company Descriptive Date': '{}/{}/{}'.format(lines[i][63:65], lines[i][65:67], lines[i][67:69]),
            'Effective Entry Date': '{}/{}/{}'.format(lines[i][69:71], lines[i][71:73], lines[i][73:75]),
            'Settlement Date': lines[i][75:78],
            'Originator Status Code': lines[i][78:79],
            'Originating DFI Identification': lines[i][79:87],
            'Batch Number': lines[i][87:94]
        }, ignore_index=True)





entries_df = pd.DataFrame(columns=['Record Type', 
                                   'Transaction Code', 
                                   'Receiving DFI Identification', 
                                   'Check Digit', 'DFI Account Number', 
                                   'Amount', 'Individual ID Number', 'Individual Name', 
                                   'Addenda Indicator', 'Trace Number'])

# define a dictionary to store the transaction code descriptions
transaction_code_descriptions = {'21': 'ACH Credit',
                                 '22': 'Deposit destined for a Checking Account',
                                 '23': 'ACH Credit Pre-Note',
                                 '24': 'ACH Debit Pre-Note',
                                 '25': 'ACH Credit Reversal',
                                 '26': 'ACH Debit Reversal',
                                 '27': 'Debit destined for a Checking Account',
                                 '28': 'ACH Debit Reversal - NSF',
                                 '29': 'ACH Credit Correction',
                                 '31': 'ACH Savings Deposit',
                                 '32': 'Deposit destined for a Savings Account',
                                 '33': 'ACH Savings Payment',
                                 '34': 'ACH Savings Receipt',
                                 '35': 'ACH General Ledger Credit',
                                 '36': 'ACH General Ledger Debit',
                                 '37': 'ACH Loan Credit',
                                 '38': 'ACH Loan Debit'}

# loop through each line in the file and extract the relevant fields
for i in range(1, len(lines) - 1, 1):
    if lines[i][0:1] == '6':
        transaction_code = lines[i][1:3]
        transaction_description = transaction_code_descriptions.get(transaction_code, 'Invalid Transaction Code')
        entries_df = entries_df.append({
            'Record Type': lines[i][0:1],
            'Transaction Code': '{}: {}'.format(transaction_code, transaction_description),
            'Receiving DFI Identification': lines[i][3:11],
            'Check Digit': lines[i][11:12],
            'DFI Account Number': lines[i][12:29],
            'Amount': '${:.2f}'.format(float(lines[i][29:39])/100),
            'Individual ID Number': lines[i][39:54],
            'Individual Name': lines[i][54:76],
            'Discretionary Data': lines[i][76:78],
            'Addenda Indicator': lines[i][78:79],
            'Trace Number': lines[i][79:94]
        }, ignore_index=True)



# Initialize Batch Control Record table
# Batch Control Record
# Initialize Batch Control Record table
# Batch Control Record
batch_control_df = pd.DataFrame(columns=['Record Type Code', 'Service Class Code', 'Entry/Addenda Count', 'Entry Hash',
                                          'Total Debit Amount', 'Total Credit Amount', 'Company Id',
                                          'Message Authentication Code', 'Reserved','Originating DFI Identification','Batch Number', 'Receiving DFI Identification Sum'])

batch_count = 0
receiving_dfi_sum = 0

for line in lines:
    record_type = line[0]
    if record_type == '5':
        batch_count += 1
    elif record_type == '6':
        receiving_dfi_sum += int(line[3:11])

    elif record_type == '8':
        service_class_code = line[1:4]
        if service_class_code == '200':
            service_class_code += ':ACH Entries Mixed Debits and Credits'
        elif service_class_code == '220':
            service_class_code += ':ACH Credits Only'
        elif service_class_code == '225':
            service_class_code += ':ACH credits originated by the FI, addenda only'
        elif service_class_code == '280':
            service_class_code += ':ACH debits originated by the FI'
        elif service_class_code == '500':
            service_class_code += ':ACH debits originated by the FI, mixed addenda'
        elif service_class_code == '505':
            service_class_code += ':ACH debits originated by the FI, addenda only'
        elif service_class_code == '225':
            service_class_code += ':ACH credits originated by the FI, addenda only'
        elif service_class_code == '225':
            service_class_code += ':ACH credits originated by the FI, addenda only'
        elif service_class_code == '225':
            service_class_code += ':ACH credits originated by the FI, addenda only'
        
        batch_control_df = batch_control_df.append({
            'Record Type Code': '8',
            'Service Class Code': service_class_code,
            'Entry/Addenda Count': line[4:10],
            'Entry Hash': line[10:20],
            'Total Debit Amount':'${:.2f}'.format(float( line[20:31])/100),
            'Total Credit Amount':'${:.2f}'.format(float( line[31:44])/100),
            'Company Id': line[44:54],
            'Message Authentication Code': line[54:73],
            'Reserved': line[73:79],
            'Originating DFI Identification':line[79:87],
            'Batch Number':line[87:94],
            'Receiving DFI Identification Sum': '${:.2f}'.format(receiving_dfi_sum/100)

        }, ignore_index=True)

        receiving_dfi_sum = 0

        if batch_count == 1:
            batch_control_df.iloc[-1, 0] = '8'
            batch_count = 0

# Remove the last row if it is empty
if batch_control_df.iloc[-1].isnull().all():
    batch_control_df = batch_control_df.iloc[:-1]
'''
total_receiving_dfi_sum = batch_control_df['Receiving DFI Identification Sum'].str.replace('$', '').str.replace(',', '').astype(float).sum()
print('Total Receiving DFI Identification Sum:', total_receiving_dfi_sum)
'''
# Convert the float to an integer and slice the last two characters

#print('Total Receiving DFI Identification Sum: ${:,.2f}'.format(total_receiving_dfi_sum))

total_receiving_dfi_sum = int(batch_control_df['Receiving DFI Identification Sum'].str.replace('$', '').str.replace(',', '').astype(float).sum() * 100)

for line in lines:
    if line.startswith('90'):
        # Parse the line to extract the required fields
        record_type = line[0:1]
        batch_count = line[1:7]
        block_count = line[7:13]
        entry_addenda_count = line[13:21]
        entry_hash = line[21:31]
        total_receiving_dfi_sum = '{:09.0f}'.format(total_receiving_dfi_sum)
        total_debit_entry_dollar_amount = '${:.2f}'.format(float(line[32:43])/100)
        total_credit_entry_dollar_amount = '${:.2f}'.format(float(line[43:55])/100)
        reserved = line[55:94]
        
        # Create a DataFrame with the parsed fields
        file_control_df = pd.DataFrame([{
            'Record Type': record_type,
            'Batch Count': batch_count,
            'Block Count': block_count,
            'Entry/Addenda Count': entry_addenda_count,
            'Entry Hash': entry_hash,
            'Total Receiving DFI Identification Sum': total_receiving_dfi_sum,
            'Total Debit Entry Dollar Amount': total_debit_entry_dollar_amount,
            'Total Credit Entry Dollar Amount': total_credit_entry_dollar_amount,
            'Reserved': reserved
        }])
        
        # Compare the Entry Hash with Total Receiving DFI Identification Sum to validate the ACH file
        if entry_hash == str(total_receiving_dfi_sum).zfill(10):
          validity = "Valid"
        else:
          validity = "Not Valid"

# Create the table with the output
        table = [["Entry Hash", entry_hash],
         ["Total Receiving DFI Identification Sum", total_receiving_dfi_sum],
         ["Validity", validity]]

# Print the table
        print(tabulate(table, headers=["Calculation", "Value"],tablefmt='fancy_grid'))









'''
from decimal import Decimal


# Convert all monetary values to Decimal type
#batch_control_df['total_credit_amount'] = batch_control_df['total_credit_amount'].apply(lambda x: Decimal(x.replace('$','')))
#file_control_df['total_credit_amount'] = file_control_df['total_credit_amount'].apply(lambda x: Decimal(x.replace('$','')))

total_credit_amount = 0.0
for index, row in batch_control_df.iterrows():
    credit_amount = float(row['Total Credit Amount'].replace('$', '').replace(',', ''))
    total_credit_amount += credit_amount

batch1_credit_amount = float(batch_control_df.loc[0, 'Total Credit Amount'].replace('$', '').replace(',', ''))
batch2_credit_amount = float(batch_control_df.loc[1, 'Total Credit Amount'].replace('$', '').replace(',', ''))
sum_credit_amount = batch1_credit_amount + batch2_credit_amount

if not file_control_df.empty:
    file_control_credit_amount = float(file_control_df.iloc[0]['Total Credit Entry Dollar Amount'].replace('$', '').replace(',', ''))
else:
    file_control_credit_amount = 0

if total_credit_amount == file_control_credit_amount:
    ach_validity = 'Valid'
else:
    ach_validity = 'Invalid'
'''

# Define the expected record types
valid_header = '1'
valid_batch = '5'
valid_file_control = '9'

if lines[0].startswith(valid_header):
    # Check if file ends with a valid file control record
    if lines[-1].startswith(valid_file_control):
        # Check if file contains at least one batch record
        if any(line.startswith(valid_batch) for line in lines):
            file_validity = "Valid"
        else:
            file_validity = "Invalid"
    else:
        file_validity = "Invalid"
else:
    file_validity = "Invalid"


ach_table_headers = ['ACH File','Result']
ach_table_data = [[f.name ,file_validity]]
print(tabulate(ach_table_data, headers=ach_table_headers, tablefmt='fancy_grid'))
    

# Print results in tabular format
table_headers = ['Batch Details', 'Credit Amount']
table_data = [['Batch 1', batch1_credit_amount],
              ['Batch 2', batch2_credit_amount],
              ['Sum of Batch 1 and Batch 2', sum_credit_amount],
              ['Total Credit Amount', file_control_credit_amount],
              ['ACH Validity', ach_validity]]

print(tabulate(table_data, headers=table_headers, tablefmt='fancy_grid'))


print("\n Header Record: \n")
print(tabulate(header_df, headers='keys', tablefmt='fancy_grid'))

# Print the batch header DataFrame
print("\nBatch Header Record: \n")
print(tabulate(batch_header_df, headers='keys', tablefmt='fancy_grid'))

# Print the entries DataFrame
print("\nEntries: \n")
print(tabulate(entries_df, headers='keys', tablefmt='fancy_grid'))

print("\nBatch Control Record: \n")
print(tabulate(batch_control_df, headers='keys', tablefmt='fancy_grid'))

print("\nFile Control Record: \n")
print(tabulate(file_control_df, headers='keys', tablefmt='fancy_grid'))

'''
import pandas as pd

# Concatenate the dataframes into a single dataframe
all_data_df = pd.concat([header_df, batch_header_df, entries_df, batch_control_df, file_control_df], ignore_index=True)

# Create a pandas Excel writer using the xlsxwriter engine
writer = pd.ExcelWriter('output1.xlsx', engine='xlsxwriter')

# Write the data to a single sheet in the Excel file
all_data_df.to_excel(writer, sheet_name='Data', index=False)

# Close the Excel writer
writer.save()

# Print confirmation message
print('Data saved to output1.xlsx')
'''

import pandas as pd

# Create a pandas Excel writer using the xlsxwriter engine
writer = pd.ExcelWriter('ACH_Exported_File.xlsx', engine='xlsxwriter')

# Define cell formats
header_format = writer.book.add_format({'bg_color': '#DDEBF7', 'bold': True, 'border': 1})
data_format = writer.book.add_format({'border': 1})

# Write header_df to Excel
header_df.to_excel(writer, sheet_name='Header', index=False, startrow=0, startcol=0, header=False)
worksheet = writer.sheets['Header']
for i, col in enumerate(header_df.columns):
    worksheet.write(0, i, col, header_format)
    for j, value in enumerate(header_df[col]):
        worksheet.write(j+1, i, value, data_format)

# Write batch_header_df to Excel
batch_header_df.to_excel(writer, sheet_name='Batch Header', index=False, startrow=0, startcol=0, header=False)
worksheet = writer.sheets['Batch Header']
for i, col in enumerate(batch_header_df.columns):
    worksheet.write(0, i, col, header_format)
    for j, value in enumerate(batch_header_df[col]):
        worksheet.write(j+1, i, value, data_format)

# Write the rest of the dataframes to Excel in separate tables
entries_df.to_excel(writer, sheet_name='Entries', index=False, startrow=0, startcol=0, header=True)
worksheet = writer.sheets['Entries']
for i, col in enumerate(entries_df.columns):
    worksheet.write(0, i, col, header_format)
    for j, value in enumerate(entries_df[col]):
        worksheet.write(j+1, i, value, data_format)

batch_control_df.to_excel(writer, sheet_name='Batch Control', index=False, startrow=0, startcol=0, header=True)
worksheet = writer.sheets['Batch Control']
for i, col in enumerate(batch_control_df.columns):
    worksheet.write(0, i, col, header_format)
    for j, value in enumerate(batch_control_df[col]):
        worksheet.write(j+1, i, value, data_format)

file_control_df.to_excel(writer, sheet_name='File Control', index=False, startrow=0, startcol=0, header=True)
worksheet = writer.sheets['File Control']
for i, col in enumerate(file_control_df.columns):
    worksheet.write(0, i, col, header_format)
    for j, value in enumerate(file_control_df[col]):
        worksheet.write(j+1, i, value, data_format)

# Save the Excel file and close the Pandas Excel writer
writer.save()
print('\n Data from ACH file Exported successfully and saved to ACH Exported File.xlsx \n')

'''
with open('SampleNacha.ach', 'r') as f:
    lines = f.readlines()

header = lines[0:1][0]
batch_headers = [l for l in lines[1:] if l[0:1] == '5']
batch_headers.append(lines[-1])

# Extract WEB and TEL Entry, PPD, CCD, and WEB Addenda records
web_tel_entries = [l for l in lines if l[0:1] == '6' and l[10:13] == 'WEB' or l[10:13] == 'TEL']
ppd_records = [l for l in lines if l[0:1] == '6' and l[10:13] == 'PPD']
ccd_records = [l for l in lines if l[0:1] == '6' and l[10:13] == 'CCD']
web_addenda_records = [l for l in lines if l[0:1] == '7' and l[1:3] == '05']

# Print header table
header_table = []
header_table.append(['Field Name', 'Value'])
header_table.append(['File ID Modifier', header[1:2]])
header_table.append(['Immediate Destination', header[3:13]])
header_table.append(['Immediate Origin', header[13:23]])
header_table.append(['File Creation Date', header[23:29]])
header_table.append(['File Creation Time', header[29:33]])
header_table.append(['File ID', header[33:37]])
header_table.append(['Record Size', header[37:39]])
header_table.append(['Blocking Factor', header[39:41]])
header_table.append(['Format Code', header[41:43]])
header_table.append(['Immediate Destination Name', header[43:63]])
header_table.append(['Immediate Origin Name', header[63:83]])
header_table.append(['Reference Code', header[83:93]])
print('\nHeader Table:\n')
for row in header_table:
    print('{:<30}{}'.format(row[0], row[1]))

# Print batch header table
for i, bh in enumerate(batch_headers):
    bh_table = []
    bh_table.append(['Field Name', 'Value'])
    bh_table.append(['Service Class Code', bh[1:4]])
    bh_table.append(['Company Name', bh[4:20]])
    bh_table.append(['Company Discretionary Data', bh[20:40]])
    bh_table.append(['Company Identification', bh[40:50]])
    bh_table.append(['Standard Entry Class Code', bh[50:53]])
    bh_table.append(['Company Entry Description', bh[53:63]])
    bh_table.append(['Company Descriptive Date', bh[63:68]])
    bh_table.append(['Effective Entry Date', bh[68:74]])
    bh_table.append(['Settlement Date', bh[74:80]])
    bh_table.append(['Originator Status Code', bh[80:81]])
    bh_table.append(['Originating DFI Identification', bh[81:91]])
    bh_table.append(['Batch Number', bh[91:94]])
    print('\nBatch {} Header Table:\n'.format(i+1))
    for row in bh_table:
        print('{:<30}{}'.format(row[0], row[1]))

# Print WEB and TEL Entry
print("\nWEB and TEL Entry:")
print("-" * 55)
for i, record in enumerate(records):
    if record['record_type'] == '6':
        entry = record['entry_detail']
        if entry['transaction_code'] in ['27', '28']:
            print(f"{i+1}. {entry['originating_dfi_identification']}"
                  f" {entry['DFI_account_number']} {entry['amount']:0.2f}")
print()

# Print PPD
print("PPD:")
print("-" * 55)
for i, record in enumerate(records):
    if record['record_type'] == '6':
        entry = record['entry_detail']
        if entry['transaction_code'] == '22':
            print(f"{i+1}. {entry['identification_number']} "
                  f"{entry['individual_id_number']} {entry['amount']:0.2f}")
print()

# Print CCD
print("CCD:")
print("-" * 55)
for i, record in enumerate(records):
    if record['record_type'] == '6':
        entry = record['entry_detail']
        if entry['transaction_code'] == '23':
            print(f"{i+1}. {entry['identification_number']} "
                  f"{entry['individual_id_number']} {entry['amount']:0.2f}")
print()

# Print WEB Addenda records
print("WEB Addenda records:")
print("-" * 55)
for i, record in enumerate(records):
    if record['record_type'] == '7':
        addenda = record['addenda_record']
        if addenda['addenda_type_code'] == '05':
            print(f"{i+1}. {addenda['payment_related_information']}")
print()

# Print WEB Addenda records
print("TEL Addenda records:")
print("-" * 55)
for i, record in enumerate(records):
    if record['record_type'] == '7':
        addenda = record['addenda_record']
        if addenda['addenda_type_code'] == '05':
            print(f"{i+1}. {addenda['payment_related_information']}")
print()
'''

<ipython-input-71-8f2395a3c487>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  batch_header_df = batch_header_df.append({
<ipython-input-71-8f2395a3c487>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  batch_header_df = batch_header_df.append({
<ipython-input-71-8f2395a3c487>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  batch_header_df = batch_header_df.append({
<ipython-input-71-8f2395a3c487>:113: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  entries_df = entries_df.append({
<ipython-input-71-8f2395a3c487>:113: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

╒════════════════════════════════════════╤════════════╕
│ Calculation                            │ Value      │
╞════════════════════════════════════════╪════════════╡
│ Entry Hash                             │ 0733790368 │
├────────────────────────────────────────┼────────────┤
│ Total Receiving DFI Identification Sum │ 732790368  │
├────────────────────────────────────────┼────────────┤
│ Validity                               │ Not Valid  │
╘════════════════════════════════════════╧════════════╛
╒════════════════╤══════════╕
│ ACH File       │ Result   │
╞════════════════╪══════════╡
│ WrongNacha.ach │ Valid    │
╘════════════════╧══════════╛
╒════════════════════════════╤═════════════════╕
│ Batch Details              │ Credit Amount   │
╞════════════════════════════╪═════════════════╡
│ Batch 1                    │ 50000000000.0   │
├────────────────────────────┼─────────────────┤
│ Batch 2                    │ 40000000000.0   │
├────────────────────────────┼─────────────────┤
│ S

<ipython-input-71-8f2395a3c487>:398: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


'\nwith open(\'SampleNacha.ach\', \'r\') as f:\n    lines = f.readlines()\n\nheader = lines[0:1][0]\nbatch_headers = [l for l in lines[1:] if l[0:1] == \'5\']\nbatch_headers.append(lines[-1])\n\n# Extract WEB and TEL Entry, PPD, CCD, and WEB Addenda records\nweb_tel_entries = [l for l in lines if l[0:1] == \'6\' and l[10:13] == \'WEB\' or l[10:13] == \'TEL\']\nppd_records = [l for l in lines if l[0:1] == \'6\' and l[10:13] == \'PPD\']\nccd_records = [l for l in lines if l[0:1] == \'6\' and l[10:13] == \'CCD\']\nweb_addenda_records = [l for l in lines if l[0:1] == \'7\' and l[1:3] == \'05\']\n\n# Print header table\nheader_table = []\nheader_table.append([\'Field Name\', \'Value\'])\nheader_table.append([\'File ID Modifier\', header[1:2]])\nheader_table.append([\'Immediate Destination\', header[3:13]])\nheader_table.append([\'Immediate Origin\', header[13:23]])\nheader_table.append([\'File Creation Date\', header[23:29]])\nheader_table.append([\'File Creation Time\', header[29:33]])\nhe

In [62]:
# Calculate the total receiving DFI identification sum
total_receiving_dfi_sum = batch_control_df['Receiving DFI Identification Sum'].str.replace('$', '').str.replace(',', '').astype(float).sum()

# Print the total receiving DFI identification sum
print('Total Receiving DFI Identification Sum:', total_receiving_dfi_sum)

# Loop through each line in the file
for line in lines:
    if line.startswith('90'):
        # Extract the values from the line
        entry_hash = line[21:31]
        debit_amount = float(line[32:43])/100
        credit_amount = float(line[43:55])/100
        
        # Calculate the total amount
        total_amount = debit_amount - credit_amount
        
        # Compare the entry hash and total receiving DFI identification sum
        if entry_hash == total_receiving_dfi_sum:
            print('ACH file is valid')
        else:
            print('ACH file is invalid')
        
        # Create a dictionary for the file control record
        file_control_dict = {
            'Record Type': line[0:1],
            'Batch Count': line[1:7],
            'Block Count': line[7:13],
            'Entry/Addenda Count': line[13:21],
            'Entry Hash': entry_hash,
            'Total Receiving DFI Identification Sum': '{:.0f}'.format(total_receiving_dfi_sum),
            'Total Debit Entry Dollar Amount': '${:.2f}'.format(debit_amount),
            'Total Credit Entry Dollar Amount': '${:.2f}'.format(credit_amount),
            'Total Amount': '${:.2f}'.format(total_amount),
            'Reserved': line[55:94]
        }
        
        # Print the file control record
        print('File Control Record:')
        for key, value in file_control_dict.items():
            print(key + ':', value)
        print('') # Add a blank line for readability


Total Receiving DFI Identification Sum: 7327903.68
ACH file is invalid
File Control Record:
Record Type: 9
Batch Count: 000003
Block Count: 000009
Entry/Addenda Count: 00000075
Entry Hash: 0732790368
Total Receiving DFI Identification Sum: 7327904
Total Debit Entry Dollar Amount: $996125.09
Total Credit Entry Dollar Amount: $0.00
Total Amount: $996125.09
Reserved:                                        



<ipython-input-62-7ba6a98b4f47>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  total_receiving_dfi_sum = batch_control_df['Receiving DFI Identification Sum'].str.replace('$', '').str.replace(',', '').astype(float).sum()
